In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 
import os


In [2]:

gb = []
data_path = './dataset-dev/plates/'
for i in os.listdir(data_path):
    if i.__contains__('GB'):
        
        gb.append(i)


In [6]:
def process(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_canny = cv2.Canny(img_gray, 50, 150, apertureSize=5,L2gradient=False)
    kernel = np.ones((5, 5), np.uint8)
    img_dilate = cv2.dilate(img_canny, kernel, iterations=3)

    return img_dilate

#first phase of segmentation to remove outer border present in images
def perfect_phase_1(img_path):
    img = cv2.imread(img_path)
    img_copy = np.copy(img)
    
    processed_img = process(img_copy)

    contours, _ = cv2.findContours(processed_img, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
    
    filter_contours = [ct for ct in contours if cv2.contourArea(ct) > 5000] #filter out small contours
    areas = []
    
    
    for ct in filter_contours:
        x, y, w, h = cv2.boundingRect(ct)
        areas.append(w * h)
        
    #get the two biggest contours
    top2 = []
    for i in range(3):
        largest_index = areas.index(max(areas))
        largest_contour = filter_contours[largest_index]    
        top2.append(largest_contour)
        
        # Remove the selected contour from the lists
        del areas[largest_index]
        del filter_contours[largest_index]
    

    big = cv2.contourArea(top2[0])
    small = cv2.contourArea(top2[1]) + cv2.contourArea(top2[2])
    ratio = 0
    if cv2.contourArea(top2[1]) >= cv2.contourArea(top2[2]):
        ratio = cv2.contourArea(top2[2]) / cv2.contourArea(top2[1])
    else:
        ratio = cv2.contourArea(top2[1]) / cv2.contourArea(top2[2])
    


    
    if small/big >0.9 and ratio >0.7:

        top2 = top2[1:]
    else:
        top2 = [top2[1]]
    
    

    color =[255,255,255]

    res = []
    for ct in top2:
        new_image = np.zeros(img.shape).astype(np.uint8)

        epsilon = 0.01 * cv2.arcLength(ct, True)
        approx = cv2.approxPolyDP(ct, epsilon, True)
        hull = cv2.convexHull(approx)

        cv2.drawContours(img_copy, [hull], -1, (255, 0, 0), 4)
        
        
        cv2.fillPoly(new_image, pts =[hull], color=color)
        
        res.append(cv2.bitwise_and(img_copy,new_image))
        
        
    #cv2.fillPoly(mask2,top2, color=color)
    #res2 = cv2.bitwise_and(img_copy,mask2)
        #crop to hull bbox
        #print(bbox[0])
        #new_image = new_image[hull]     
        
        #cv2.drawContours(img_copy,[ct],0,(0,0,255),2)
        #x, y, w, h = cv2.boundingRect(ct)
        #cv2.rectangle(img_copy, (x, y), (x + w, y + h), (255, 0, 0), 3)
        #cv2.rectangle(img_copy, (x, y), (x + w, y + h), (0, 0, 255), 2)
    return img_copy, top2, res
    

In [7]:
for i in gb[0:1]:
    
    img,_ ,res= perfect_phase_1(data_path+i)
    cv2.imshow("phase 1", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    for mask in res:
        cv2.imshow("phase 1", mask)
        cv2.waitKey(0)
        cv2.destroyAllWindows()